<a href="https://colab.research.google.com/github/varsha116/FallDetection/blob/main/FallDetection_CollaborationWithClipModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mount Google Drive to access files
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Install necessary libraries
!pip install ultralytics deepface opencv-python
!pip install deep-sort-realtime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.9/914.9 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 62.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.5 MB/s eta 0:00:00
   ━

In [ ]:
!pip install clip-by-openai torch torchvision torchaudio

INFO: pip is looking at multiple versions of clip-by-openai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of clip-by-openai to determine which version is compatible with other requirements. This could take a while.
ERROR: Cannot install clip-by-openai==0.1.0, clip-by-openai==0.1.1, clip-by-openai==0.1.1.2, clip-by-openai==0.1.1.3, clip-by-openai==0.1.1.4, clip-by-openai==0.1.1.5, clip-by-openai==1.0.1, clip-by-openai==1.1 and torch because these package versions have conflicting dependencies.

The conflict is caused by:
    The user requested torch
    clip-by-openai 1.1 depends on torch<1.7.2 and >=1.7.1
    The user requested torch
    clip-by-openai 1.0.1 depends on torch<1.7.2 and >=1.7.1
    The user requested torch
    clip-by-openai 0.1.1.5 depends on torch==1.7.1
    The user requested torch
    clip-by-openai 0.1.1.4 depends on torch==1.7.1
    The user requested torch
    clip-by-open

In [ ]:
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-qjlay5xu
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-qjlay5xu
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
  Using cached ftfy-6.3.1-py3-none-any.whl.metadata (7.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.8 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369489 sha256=992eb54b21e17e41917c3bfd441dbfadfd07524fc603ea027586e5ecca250e2b
  Stored in directory: /tmp/pip-ephem-wheel-cache-u5_giyi_/wheels/3f/7c/a4/9b490845988bf7a4db33674d52f709f088f64392063872eb9a
Successfully built clip


In [ ]:
pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 31.4 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.1
    Uninstalling transformers-4.47.1:
      Successfully uninstalled transformers-4.47.1


In [ ]:
!pip install --upgrade transformers requests

In [ ]:
import torch
import cv2
import numpy as np
from transformers import CLIPProcessor, CLIPModel
from ultralytics import YOLO  # For person detection

# Load YOLOv8 person detection model
yolo_model = YOLO("yolov8n.pt")  # Load YOLOv8 nano for efficiency

clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32", force_download=True)
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32", force_download=True)

# Function to classify fall using CLIP
def is_falling(person_crop):
    inputs = clip_processor(
        text=["a person falling", "a person standing"],
        images=person_crop,
        return_tensors="pt",
        padding=True
    )#.to(device)

    outputs = clip_model(**inputs)
    logits_per_image = outputs.logits_per_image
    probs = logits_per_image.softmax(dim=1)
    return probs[0][0].item()  # Probability of "a person falling"

# Load video
video_path = "/content/drive/MyDrive/Mirasys/fall.mp4"  # Change to your path
cap = cv2.VideoCapture(video_path)

# Get video properties
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Save output video
output_path = "/content/drive/MyDrive/Mirasys/msys04.02.25/1op_fall_withBB_detected.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLOv8 for person detection
    results = yolo_model(frame, stream=True)

    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # Bounding box coordinates
            confidence = box.conf[0].item()  # Confidence score
            cls = int(box.cls[0].item())  # Class index

            if cls == 0 and confidence > 0.5:  # Only process 'person' detections
                person_crop = frame[y1:y2, x1:x2]  # Crop person region

                # Convert crop to RGB for CLIP
                person_crop_rgb = cv2.cvtColor(person_crop, cv2.COLOR_BGR2RGB)

                # Predict fall probability using CLIP
                fall_probability = is_falling(person_crop_rgb)

                # Classify as falling if probability >= 0.5
                label = "FALL DETECTED" if fall_probability >= 0.5 else "No Fall"
                color = (0, 0, 255) if fall_probability >= 0.5 else (0, 255, 0)

                # Draw bounding box around detected person
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(frame, f"{label} ({fall_probability:.2f})", (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

    # Write frame to output video
    out.write(frame)

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]


0: 384x640 1 person, 499.0ms
Speed: 6.7ms preprocess, 499.0ms inference, 4.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 536.1ms
Speed: 6.5ms preprocess, 536.1ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 458.4ms
Speed: 5.0ms preprocess, 458.4ms inference, 4.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 282.5ms
Speed: 2.5ms preprocess, 282.5ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 216.5ms
Speed: 2.5ms preprocess, 216.5ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 215.5ms
Speed: 2.4ms preprocess, 215.5ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 208.6ms
Speed: 3.2ms preprocess, 208.6ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 129.8ms
Speed: 2.1ms preprocess, 129.8ms inference, 1.0ms postprocess per image 